In [ ]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from typing import Tuple

outputs_path = Path("../outputs")


def load_scenario_data(name: str | Path = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
    name = "current" if name is None else name
    scenario_path = outputs_path / name if isinstance(name, str) else name
    scenario_data_path = scenario_path / "data"

    print(f"Using Scenario path: {scenario_path}")
    trips_df: pd.DataFrame = None
    route_times = []
    elapsed_times = []
    route_points = []

    for path in scenario_data_path.iterdir():
        if not path.is_dir():
            continue

        try:
            scenario_df = pd.read_csv(path / "trips.csv", skipinitialspace=True)
            scenario_df["DIR_NAME"] = path.name

            trips_df = pd.concat([trips_df, scenario_df], ignore_index=True)

            with open(path / "results.txt", "r") as f:
                lines = f.readlines()
                route_points.append({"path_name": path.name, "route": lines[2]})
                route_times.append(float(lines[3].split()[2]))
                elapsed_times.append(float(lines[5].split()[2]))

        except Exception:
            pass

    env = ""
    with open(scenario_path / ".conf", "r") as f:
        env = "".join([l.split("#")[0] for l in f.readlines()])

    ran_at = ""
    with open(scenario_path / ".run_at", "r") as f:
        ran_at = f.readline()

    results_df = pd.DataFrame(
        {
            "ROUTE_TIMES": route_times,
            "ELAPSED_TIMES": elapsed_times,
            "ROUTE_POINTS": route_points,
        }
    )

    return (trips_df, results_df, env, ran_at)

In [ ]:
trips, route_results, env, ran_at = load_scenario_data(
    # Path.cwd().parent / "queues/2024-06-24_22:09:57/outputs/2024-06-25_01:55:14/"
)
scenario_name = "Scenario"
print("Ran At: ", ran_at)

In [ ]:
print(env)

In [ ]:
from models import ReservationStatus

trips.loc[trips.RESERVATION_STATUS == str(ReservationStatus.REJECTED)]

In [ ]:
import seaborn as sns

colors = {"ACCEPTED": "blue", "REJECTED": "red"}

# Use seaborn to create the bar plot
trips_df = trips
sns.histplot(
    data=trips_df,
    x="RESERVED_AT",
    hue="RESERVATION_STATUS",
    multiple="stack",
    palette=colors,
    shrink=0.8,
    bins=range(0, 70, 10),
)
plt.xlabel("Reserved At")
plt.ylabel("Count")
plt.title("Count vs Reserved At")
plt.show()

In [ ]:
colors = {"ACCEPTED": "blue", "REJECTED": "red"}
sns.histplot(
    data=trips_df,
    x="RESERVATION_STATUS",
    stat="percent",
    hue="RESERVATION_STATUS",
    palette=colors,
    shrink=0.8,
    bins=range(0, 61, 5),
)

plt.xlabel("Reservation Status")
plt.ylabel("%")
plt.title("Reservation Status %")
plt.show()

In [ ]:
trips_df = trips
direction_counts = trips_df["DIRECTION"].value_counts()
plt.figure(figsize=(7, 5))
direction_counts.plot(kind="bar", color=["blue", "orange"])
plt.xlabel("Direction")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.title("Count of Directions")
plt.show()

In [ ]:
def plot_kde_for(df, x: str, hue: str = None, pallete: list = None):
    sns.kdeplot(
        data=df,
        x=x,
        hue=hue,
        fill=True,
        common_norm=False,
        palette=pallete,
        alpha=0.5,
        linewidth=0,
    )


results_df = None
scenario_routes_df = route_results
scenario_routes_df["NAME"] = scenario_name
results_df = pd.concat([results_df, scenario_routes_df], ignore_index=True)

# plot_kde_for(routes_df, "ROUTE_TIMES", "NAME", ["red", "blue"])

In [ ]:
plot_kde_for(scenario_routes_df, "ROUTE_TIMES", "NAME", ["red"])
plt.title(f"Route times")
plt.legend(loc="upper left")
plt.xlabel("Route times (Minutes)")

In [ ]:
plot_kde_for(scenario_routes_df, "ELAPSED_TIMES", "NAME", ["blue"])
plt.title(f"Computation Elapsed time")
plt.xlabel("Elapsed time (Seconds)")

In [ ]:
# Tweak the bins parameter to match the shape you want, odd numbers are better
sns.histplot(
    data=scenario_routes_df,
    x="ROUTE_TIMES",
    palette=colors,
    stat="count",
    shrink=1,
    bins=13,
    common_norm=False,
    kde=True,
)

In [ ]:
import numpy as np

# Calculate the mean and variance for each column
mean = np.mean(route_results["ROUTE_TIMES"], axis=0)
variance = np.var(route_results["ROUTE_TIMES"], axis=0)

print("Mean Route time: ", mean, " minutes")
print("Route time Variance: ", variance, " minutes")

In [ ]:
from models import TripDirection

dir_name = "01_07_2024__07_15_43.986264"
route_points = (
    (
        route_results["ROUTE_POINTS"]
        .apply(lambda x: x["route"] if x["path_name"] == dir_name else None)
        .dropna()
        .values[0]
    )
    .strip()
    .split("->")
)
trips_for_dir_name = trips_df[trips_df["DIR_NAME"].apply(lambda x: x == dir_name)]
route_points.pop()
print(route_points)

route_points_iterator = iter(route_points)
prev_point = int(next(route_points_iterator))

max_x = max_y = len(route_points) / 5
curr_x = curr_y = 0

nodes = [prev_point]
next_nodes = []
colors = ["#8DB1E2"]
shapes = ["s"]
sizes = [3000]
positions = [(curr_x, curr_y)]

for curr_point in route_points_iterator:
    curr_point = int(curr_point)
    curr_x = curr_x + 1
    if curr_x > max_x:
        curr_x = curr_x % max_x
        curr_y = curr_y + 4

    trip = trips_for_dir_name[trips_for_dir_name["ID"] == curr_point]
    direction = trip["DIRECTION"].values[0]
    prev_point = curr_point

    nodes.append(curr_point)
    next_nodes.append(curr_point)
    colors.append("#FFC000" if direction == TripDirection.OUTBOUND.name else "#C4D6A0")
    shapes.append("o")
    sizes.append(1500)
    positions.append((curr_x, curr_y))

next_nodes.append(0)

nodes_df = pd.DataFrame(
    {
        "NODES": nodes,
        "NEXT_NODES": next_nodes,
        "POSITIONS": positions,
        "COLORS": colors,
        "SHAPES": shapes,
        "SIZES": sizes,
    }
)

In [ ]:
import networkx as nx

G = nx.DiGraph()
edges = list(zip(nodes_df["NODES"], nodes_df["NEXT_NODES"]))
print(edges)

G.add_nodes_from(nodes_df["NODES"])
G.add_edges_from(edges)
print(G.edges)

# Generate the spring layout
# pos = nx.spring_layout(G, k=10, iterations=100_000)
pos = nodes_df["POSITIONS"]

# Draw the graph
plt.figure(figsize=(10, 8))

options = {
    "edge_color": "gray",
    "font_size": 12,
    "font_color": "black",
    "with_labels": True,
    "connectionstyle": "arc3,rad=0.2",
    "arrowstyle": "-|>",
    "arrowsize": 20,
}

plt.figure(figsize=[20, 10])
nx.draw(
    G,
    pos,
    nodelist=nodes_df["NODES"].tolist(),
    node_color=nodes_df["COLORS"],
    node_shape="o",
    node_size=nodes_df["SIZES"],
    **options,
)

# Show the plot
plt.show()

nx.write_gml(G, Path.home() / "Downloads/simple.gml")